# Plantnet Classification - Sorbier & Orme

*Andrieu Grégoire & Gille Cyprien*

## Necessary Imports

In [ ]:
import os # for folder browsing
from tqdm.notebook import tqdm # progress bars

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
import torch.nn.functional as F # loss function
from torchvision.datasets import ImageFolder # load our dataset, which has the ImageFolder structure
from torchvision import transforms, models # data augmentation and preprocessing
import torchvision.transforms.functional as TF # used in old code for on-fecth transforms
from torchvision.utils import make_grid # to display a batch 

from PIL import Image # useful for single-image loading
import copy # model checkpoints
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.preprocessing import normalize # helps to display the confusion matrix
import pandas as pd # used to create submission.csv
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib as mpl

## Data

Our data for this competition is a subset of the Pl@ntnet dataset, containing 140,256 images from 153 species.

In [ ]:
data_dir_path = "../input/polytech-nice-data-science-course-2021/polytech/"
train_dir_path = data_dir_path + "train/"
test_dir_path = data_dir_path + "test/"

In [ ]:
# All the classes
classes = os.listdir(train_dir_path)

### Exploratory Data Analysis


Before working with our data, we need to explore it to see if there are any caveats to working with it. First, let us display a few images from the dataset.

In [ ]:
plt.figure(figsize=(18, 18)) # big figure to have enough space for 16 images
for i in range(16):
    # we pick one image from each of the first 16 classes (os.listdir uses alphabetical order)
    current_path = train_dir_path + classes[i] + "/"
    current_images = os.listdir(current_path) # all images from the selected class
    
    img = Image.open(current_path + current_images[0])
    
    plt.subplot(4, 4, i+1) # 4 rows and 4 columns
    plt.imshow(img)
    plt.title(f"Image from class {classes[i]}, size {img.size}")

plt.show()

From this first step, we can see two things : 
 - The images can be different sizes
 - The orientation of the plant is relatively random, which means we can use rotations for data augmentation later.

Now, let us examine the repartition of images between classes.

In [ ]:
# compute the number of images per class
nums = {}
for plant_num in classes:
    nums[str(plant_num)] = len(os.listdir(train_dir_path + plant_num))

img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["Number of images"])

In [ ]:
# display as an histogram the size of each class in our dataset

class_numbers = [i for i in range(153)]
plt.figure(figsize=(16, 6))
plt.bar(class_numbers, [nums[str(i+1)] for i in class_numbers], width=0.4)
plt.xlim(-2, 154)
plt.xlabel('Classes')
plt.ylabel('Quantity of images available')
plt.show()

We can see that our dataset is heavily unbalanced. This is confirmed when we look at its statistics :

In [ ]:
img_per_class.describe()

Looking at the graph and at the median number of images in a class, we can see that our dataset has mostly small classes, with around 400 images, but also contains a few plant types that are widely over-represented, with over 5000 images for some.

We will absolutely have too keep this in mind, because our network will naturally predict the oversized classes more frequently. If real-life (i.e. test) data is also unbalanced in the same way, then it would be fine to leave this bias in our model. However, if the test dataset is balanced, or unbalanced differently than the train dataset, then our model would perform poorly. 

Mitigating the unbalanced-ness of the train dataset will make our model more robust and accurate (cf Progress), so that is what we will focus on in this next section.

### Data augmentation and oversampling



Our way of dealing with our unbalanced dataset is through a technique called **Oversampling**. 
During training, our dataloader will pick from under-represented classes with a higher probability. 
This is achieved through passing a weighted random sampler to the data loader : in the sampler, each element of the training set has a weight associated with it that translates to how likely the sampler is to fetch this particular element.

To ensure that our oversampling achieves our goal of balancing the classes, we determine the weights as follows:

For a sample $x$ that belongs to class $Y$:
$$
weight(x) = \frac{1}{size(Y)}
$$

That way, each individual sample in an under-represented class has a higher chance of being fetched, but samples from over-represented classes compensate this by being more numerous.

However, we now have a new problem. With our oversampling technique, samples from small classes will be used for training multiple times per epoch. This will obvioulsy affect our model's ability to generalise, hence the need for another data processing technique: **data augmentation**.

We use data augmentation directly on the input of ImageFolder: all images will pass through a set of transforms (cf. comments in the code) before being fed to our model. Some of those transforms have a certain probability of happening, which means that the network will no longer train on the same oversampled images.

As a sidenote, our validation data does not need to be augmented, but it is easier and cleaner to pass the transforms to the dataset at its creation (before the split). That is not a problem, since the purpose of validation data is merely to give us an indicative measure of how good our model is at generalising.

We may have fixed the balance in our dataset for training, but our actual dataset still "physically" has very small classes. That means that if we are not careful, while performing the split to get training and validation data, we could end up with a class being completely distributed in only the validation set. To avoid this, instead of using pytorch's `random_split` function, we use scikit learn's `train_test_split` and its `stratify` argument.

In [ ]:
# Data augmentation and normalization for training
# Just normalization for evaluation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224), # Data augmentation : crop to correct size, using a random part of the image
        transforms.RandomHorizontalFlip(), # perform a Hflip 50% of the time
        transforms.RandomVerticalFlip(), # perform a Vflip 50% of the time
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224), # not necessary for square images, can be commented out
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
# The normalization values are defined directly as such in the code of the models we will be using later, so we have to use these ones specifically.
# Same goes for input size, alexnet vgg and resnet have an input size of 224x224 (this means that we are upscaling some of our images!)

In [ ]:
train_proportion = 0.8

print("Initializing Datasets and Dataloaders...")
dataset = ImageFolder(train_dir_path, data_transforms["train"])


# split indices
train_indices, valid_indices, _, _ = train_test_split(
    range(len(dataset)),
    dataset.targets,
    stratify=dataset.targets, # This ensures that each class gets split into train/val
    test_size=1 - train_proportion
)

# split le dataset avec les indices
train_split = Subset(dataset, train_indices)
valid_split = Subset(dataset, valid_indices)

In [ ]:
# As presented above, we compute what weight each class should give to its samples
# Note : We use the nums dictionnary created earlier, which means the len(class) are those from the original, non-split dataset.
# This does not affect our sampler however, thanks to the stratify parameter we used during splitting:
# for all classes, len(class_in_train_set) = train_proportion * len(class_pre_split)
# Therefore, each class weight would simply be scaled up by 1/0.7, which does would not change anything to the relative magnitudes of the weights.
class_weights = {int(class_name):1/class_len for (class_name, class_len) in nums.items()}

In [ ]:
# Store the weight for each sample in the train set.
# This can take a lot of time, but we cannot easily do it in parallel 
# because order HAS to be consistent between the actual samples in the set and their weights

sample_weights = []
for i in tqdm(range(len(train_split))):
    _, y = train_split.__getitem__(i)
    sample_weights.append(class_weights[y+1])

In [ ]:
batch_size = 8 # Between 8 and 16 gives the best results without taking *too* long

# num workers = 2 because Kaggle CPUs have two cores (2-core Intel(R) Xeon(R) CPU @ 2.30GHz)

# Creating the sampler to fix our unbalanced dataset
# Note : We can technically make our training data as big as we want now with this way of sampling
# for example with 
#               WeightedRandomSampler(sample_weights, 3 * len(sample_weights), replacement=True)
# but then training takes too long
train_sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)


# Training and validation dataloaders
train_dl = DataLoader(train_split, batch_size=batch_size, sampler=train_sampler, num_workers=2)
val_dl = DataLoader(valid_split, batch_size=batch_size, num_workers=2)

In [ ]:
# display one training batch
for images, labels in train_dl:
    _, ax = plt.subplots(figsize=(30, 30))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))
    break

In [ ]:
# If we have a GPU, set the device we will be using to gpu
# using the GPU to host the model and process batches in parallel makes training much faster
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# helper function to move data to the GPU correctly 
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

## Network

For more details on the other models that we tried before settling on the one used below, see the Progress section.

For our task, we use the pre-trained resnet152 model, which is present directly in the torchvision.models module. Before instantiating it however, we define a few wrapper classes for ease of use and display later.

In [ ]:
# helper function to score a prediction vector
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


# general model methods
class PlantNetModel(nn.Module):

    def batch_training(self, batch):
        """feed a batch to the model, compute and return the cross entropy loss"""
        imgs, labels = batch
        out = self(imgs)
        loss = F.cross_entropy(out, labels)
        return loss
    
    def batch_evaluation(self, batch):
        """feed a batch to the model, 
        compute the cross entropy loss and the accuracy, 
        and return them in a dict"""
        imgs, labels = batch
        out = self(imgs)
        loss = F.cross_entropy(out, labels)
        acc = accuracy(out, labels)
        # note: we only compute the accuracy during evaluation (and not training) to make training faster
        return {"val_loss": loss.detach(), "val_accuracy": acc}
    
    def eval_epoch(self, outputs):
        """return the mean loss and accuracy over one epoch"""
        batch_losses = [x["val_loss"] for x in outputs]
        batch_accuracy = [x["val_accuracy"] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()  
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {"val_loss": epoch_loss, "val_accuracy": epoch_accuracy}
    
    
    def print_epoch_end(self, epoch, result):
        """Helper end-of-epoch print"""
        # Epoch +1 because "Epoch 0" doesn't look good
        print((f"""Epoch [{epoch+1}], train_loss: {result['train_loss']}, val_loss: {result['val_loss']}, val_acc: {result['val_accuracy']}""")
        )

In [ ]:
class PlantResNet152(PlantNetModel):
    def __init__(self, feature_extract=True):
        super().__init__()
        self.net = self.initialize_model(feature_extract)
        self.FE = feature_extract

    def forward(self, batch):
        return self.net(batch)

    def initialize_model(self, feature_extract):
        """Helper function to instantiate the model
        And freeze weights if feature_extract is set to True"""
        model = models.resnet152()
        # if we are only doing feature extraction, freeze all weights in the model
        if feature_extract:
            for param in model.parameters():
                param.requires_grad = False
        
        # input size of the last layer of the model
        num_ftrs = model.fc.in_features
        # replacing the last layer with a new one
        # Note : the new weights thus have require_grad set to true by default
        model.fc = nn.Linear(num_ftrs, 153)

        return model

## Training

During training, we are going to fine-tune the pre-trained weights of resnet152. The training loop is very standard, and saves a checkpoint of the best model for later use.

In [ ]:
# Function that runs the evaluation phase
@torch.no_grad()
def evaluate(model, val_dl, prog_bar=False):
    model.eval() # evaluation mode
    if prog_bar:
        outputs = [model.batch_evaluation(to_device(batch, device)) for batch in tqdm(val_dl)]
    else:
        outputs = [model.batch_evaluation(to_device(batch, device)) for batch in val_dl]
    return model.eval_epoch(outputs)
    
    
def full_training(epochs, 
                  lr, 
                  model, 
                  train_dl, 
                  val_loader,
                  weight_decay=0,
                  grad_clip=None, 
                  optimizer=torch.optim.SGD
                  ):
    """Full training routine

    Parameters
    ----------
    epochs : [int]
        [The number of epochs of training]
    lr : [float]
        [Initial learning rate]
    model : [a subclass of PlantNetModel that implements the forward method]
        [model to train]
    train_dl : [DataLoader]
        [training set dataloader]
    val_loader : [DataLoader]
        [validation set dataloader]
    weight_decay : [float], optional
        [decay coefficient for the weights of the model]
    grad_clip : [float], optional
        [if specified, value to which we clip exploding gradients], by default None
    optimizer : [optimizer], optional
        [learning algorithm], by default torch.optim.SGD

    Returns
    -------
    [list]
        [losses and accuracy during training]
    """
    
    torch.cuda.empty_cache() # make space
    history = []
    best_val_acc = 0
    best_model_wts = copy.deepcopy(model.net.state_dict())

    # listing the weights to update to pass them to the optimizer
    params_to_update = model.net.parameters()
    if model.FE:
        params_to_update = []
        for _,param in model.net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)

    
    optimizer = optimizer(params_to_update, lr, weight_decay=weight_decay)
    
    # Note: we define the scheduler inside this function (unlike the optimizer for example)
    # because schedulers have very different behaviors (some have args for sched.step(), some need to be called between each batch, etc...)
    # Note 2 : There are many ways to schedule the learning rate, see the Progress section for more details on our choice
    # sched = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, mode="max", verbose=True)
    sched = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, verbose=True)
    
    for epoch in range(epochs):
        ## Training
        print(f"\nStarted training epoch {epoch+1}...")
        model.train() # training mode
        train_losses = []
        for batch in tqdm(train_dl):

            batch = to_device(batch, device) # load the batch to the GPU
            loss = model.batch_training(batch) # train on current batch
            train_losses.append(loss)
            loss.backward() # propagate loss
            
            # gradient clipping to prevent exploding gradients
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step() # update weights
            optimizer.zero_grad() # reset gradient computations for next batch

        ## validation
        # evaluation routine
        print("Evaluating...")
        result = evaluate(model, val_loader, prog_bar=False)
        # mean loss over all batches
        result['train_loss'] = torch.stack(train_losses).mean().item()
        # display the end-of-epoch stats
        model.print_epoch_end(epoch, result)
        
        history.append(result)
        
        vc = result["val_accuracy"]
        # sched.step(vc) # for reduce lr on plateau
        sched.step()
        if  vc > best_val_acc:
            best_val_acc = vc
            print(f"Saving new best model from epoch {epoch+1} with val_acc of {vc} ...")
            best_model_wts = copy.deepcopy(model.net.state_dict())
            torch.save(best_model_wts, "best_model.pt")
    
    model.net.load_state_dict(best_model_wts) # give model the best weights encountered during training
    return history

In [ ]:
## Controllable parameters

# toggle between fine-tuning and feature extraction
feature_extract = False

# Number of epochs
epochs = 12
# starting learning rate
lr = 0.00001
# Max gradient allowed
grad_clip = 5 # just in case there is a bad batch
# Decay coefficient for regularization
# weight_decay = 1e-5 # leave commented in case of fine-tuning

# optim = torch.optim.SGD
optim = torch.optim.Adam 
# optim = torch.optim.AdamW

In [ ]:
model = PlantResNet152(feature_extract=feature_extract)
model = to_device(model, device)


In [ ]:
history = [evaluate(model, val_dl, prog_bar=True)] # before any training, store the performance of the model

In [ ]:
history # since the weights are initialised randomly, the model does not perform well without training

In [ ]:
history += full_training(epochs, lr, model, train_dl, val_dl, grad_clip=grad_clip, optimizer=optim)

In [ ]:
# very simple plotting of the accuracy and losses through training
plt.figure(figsize=(15, 5))
mpl.rcParams["axes.titlesize"] = 12
mpl.rcParams["axes.labelsize"] = 15
mpl.rcParams["legend.fontsize"] = 15
mpl.rcParams["lines.linewidth"] = 3
mpl.rcParams["xtick.labelsize"] = 12
mpl.rcParams["ytick.labelsize"] = 12

accuracies = [x['val_accuracy'] for x in history]
plt.subplot(121)
plt.plot(accuracies)
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Accuracy by Epoch')

train_losses = [x.get('train_loss') for x in history]
val_losses = [x['val_loss'].cpu() for x in history]
plt.subplot(122)
plt.plot(train_losses, 'r')
plt.plot(val_losses, 'g')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy Loss')
plt.legend(['Training', 'Validation'])
plt.title('Loss by Epoch')

plt.show()

## Experiments

In [ ]:
model.eval() # we don't want to train the model anymore

### Confusion matrix

In [ ]:
# We will use the validation data to compute our confusion matrix

true_labels = []
pred_labels = []
for batch in tqdm(val_dl):
    batch = to_device(batch, device)
    imgs, labels = batch
    out = model(imgs)
    _, preds = torch.max(out, dim=1)

    # get the labels out of the tensors
    true_labels += labels.tolist()
    pred_labels += preds.tolist()



In [ ]:

def disp_confusion_matrix(y_true, y_pred, classes):

    cm = confusion_matrix(y_true=y_true, y_pred=y_pred)
    cm = normalize(cm, axis=1, norm='l1') # for the colormap

    df_cm = pd.DataFrame(cm, index = classes, columns = classes)

    plt.figure(figsize=(120,70))
    sn.heatmap(df_cm, annot=True, cmap=sn.cubehelix_palette(light=1, as_cmap=True))
    plt.title('Confusion Matrix',fontdict={'fontsize':20})
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()


disp_confusion_matrix(true_labels, pred_labels, dataset.classes)

It seems that some classes are very rarely correctly predicted. Going back to the histogram at the start of this notebook, it seems that they are all classes with very small cardinality : oversampling and data augmentation has its limits. On top of that, some classes are very often mistaken for another specific class, slightly bigger. Those pairs of classes represent very similar plant types, which the model cannot differentiate. This phenomenon can be mitigated with a weighted loss function, but overall accuracy does not improve along with it (Cf. Progress).

### Creating challenge submission

In [ ]:
import pandas as pd

# import the sample submission to imitate its structure
orig_sub = pd.read_csv(f"{data_dir_path}sample_submission.csv")

In [ ]:
def pred(img_name):
    """Get the prediction for a single image of the test folder"""
    img = Image.open(test_dir_path + img_name)
    in_tr = data_transforms["val"] # input transform
    img = in_tr(img)
    
    img_batch = img.unsqueeze(0).cuda() # add the batch dimension
    out = model(img_batch)
    _, pred_class = torch.max(out, dim=1)
    return pred_class

In [ ]:
submission = pd.DataFrame(columns=["image_name", "class"])

for i, img_name in enumerate(tqdm(orig_sub["image_name"])):
    submission.at[i, "image_name"] = img_name
    submission.at[i, "class"] = dataset.classes[pred(img_name).item()]

In [ ]:
submission.to_csv("submission.csv", index=False)

## Progress

In this section we will go through our different reasonings and tests behind all the changes that lead to the final state of the notebook above, chronologically.

### First trials : ConvNet

At first, to get a baseline for what a "good" performance should look like, we wanted to try a very simple run of a very simple home-made convolutional network.

|*First trial*| val_acc : 0.42 |
|---|:---:|
|Architecture|ConvNet (see Annex)|
|input_size|183x183|
|dropout|0.1|
|train_proportion|0.9|
|splitting method|torch.utils.data.random_split|
|batch_size|32|
|epochs|5|
|lr|0.01|
|grad_clip|No|
|weight_decay|No|
|optimizer|Adam|
|lr scheduler|No|

Note: the input size is 183x183 because we noticed that that was the smallest size present in the dataset, and we wanted to avoid upscaling images if we did not have to.

This first run and the experiments we did around it allowed us to establish a few things:
 - Adam seemed to be the best optimizer, with the quickest convergence and best scores.
 - 0.01 was the starting learning rate with the best results.
 - However, the model's performance started stagnating very quickly: we need to adjust the learning rate after a few epochs.
 - We could afford to reduce the training proportion without losing too much accuracy, to speed up training.
 - A batch size of 8 or 16 actually gave better results than higher ones, while only adding a few minutes to training.
 - A probability of dropout of 0.1 yielded the best results.
 - There is something wrong with our submission : our validation accuracy is 0.42, but our test accuracy is around 0.005.

This last point would be our main issue for a while. At the time, our main hypothesis was that the test set had a very different class distribution than the training dataset, on purpose, to force us to have very robust models. We did the EDA part of this notebook, and thought that maybe the test set was balanced. In fact, there were a few classes that our current model never predicted (the small ones), so we decided to put our hypothesis to the test.

### Main trials I : ResNet with no Data Augmentation

To see if our submission accuracy issue really came from a dataset difference, we decided first to try to improve our model's performances, to see whether the test accuracy followed the validation accuracy or not.

With that goal in mind, we used a much more complex home-made model: a residual convolutional network. Seeing how similar some of the classes were, we wanted to make sure that we had a model complex enough to achieve our complex classification task.

So, after a lot of (some failed, some successful) runs, tuning of the parameters to get the best validation accuracy possible without modifying the dataset in any way, we ended up with this state of our model:

|*Result of the main trials*| val_acc : 0.56 |
|---|:---:|
|Architecture|ResNet (see Annex)|
|input_size|183x183|
|dropout|0.1|
|train_proportion|0.75|
|splitting method|sklearn.model_selection.train_test_split|
|batch_size|16|
|epochs|15|
|lr|0.01|
|grad_clip|1|
|weight_decay|1e-5|
|optimizer|Adam|
|lr scheduler|StepLR with step_size = 4|

Those several runs and the experiments we did around them allowed us to establish more things:
 - Making our ResNet deeper did not have any noticeable impact on either validation accuracy nor test accuracy.
 - Using more epochs, or doing a warm restart on our trained model (i.e. recycling through the learning rate) did not improve the validation accuracy : it seemed like we reached a cap.
 - Keeping train proportion between 0.75 and 0.8 was the best tradeoff between performance and training speed.
 - As mentioned above in the code, using train_test_split instead of random_split guaranteed properly stratified train and val datasets.
 - Using StepLR performed better than both ReduceLROnPlateau using the validation accuracy, and CycleLR (increasing batch-by-batch the lr until around a third of the epochs, then going back down to a very low value).
 - Due to the residual nature of our network, some training loops displayed wild divergence of the network because of the exploding gradient problem. Using gradient clipping with values between 1 and 5 fixed that issue without slowing down convergence.
 - Weight decay improved very marginally the performances during training, but had no visible drawback.
 - Kaggle runs and sessions time out after 9 hours of runtime, so it is wise to save checkpoints of halfway-trained models of we intend to use them afterwards.
 - Our submission definitely has an issue, because it does not seem to improve even though the model itself has a better validation accuracy.

At this point, our conclusion was that we had to balance our training dataset one way or another. We blamed the inbalance in the dataset for both our validation accuracy cap, and our abysmal test score.

### Main trials II : ResNet with Oversampling, and Data Augmentation

We had a few options to balance our dataset. 

The first one, the brute-force one, was to physically save augmented images (rotated by some multiple of 90 degrees, lightly blurred, etc) to increase the size of each class until it reached a given threshold. There were several issue with this approach. First, determining the threshold wouldn't be easy: some classes only have around 100 samples, while the largest one has around 6000. Data augmentation has its limits before it starts to be detrimental to , and it would be unreasonable to try to increase each class to a size even a third as large as the largest one. Second, the input folder on Kaggle is read-only, and it would not be practical to have our augmented dataset be split between the input and output folders (especially since we would have to re-do the augmentation everytime, seeing as the output folder is empty at the start of every session). In short, this solution seems to be both limited in effect and cumbersome in practice. (Cf. Annex : Manual dataset balancing)

The second one was to leave the dataset intact, and use a weighted sampler to oversample the under-represented classes, and conversely undersample the over-representated classes (Cf. comments about that technique above in the code). Paired with random data augmentation on every sample, this proved to be quite effective.

The third one was to use a weighted loss, where smaller classes would have a greater impact on the loss than larger ones. Whether paired with oversampling and data augmentation or not, this actually worsened our score. However, upon inspection of the confusion matrix, it had the merit of making very small classes have many more correct predictions than before. Sadly, this improvement was largely outweighed by how worse predictions on large classes were doing.

After even more 7-hour-long runs, tuning our new parameters, we ended up with this state of our experiments:

|*Result of the main trials with oversampling and DA*| val_acc : 0.62 |
|---|:---:|
|Architecture|ResNet (see Annex)|
|input_size|183x183|
|dropout|0.1|
|train_proportion|0.8|
|splitting method|sklearn.model_selection.train_test_split|
|Augmentation technique|All classes, random hflip, vflip, blur (Cf. Annex: RandomFlipsTransform)|
|len(train_dataloader)|2*len(train_split)|
|batch_size|16|
|epochs|10|
|lr|0.01|
|grad_clip|1|
|weight_decay|1e-5|
|optimizer|Adam|
|lr scheduler|ReduceLROnPlateau with patience=2|

Those several runs and the experiments we did around them allowed us to conclude some points:
 - At first, we tried to perform augmentation only on small classes, using a custom dataset (Cf. Annex: SmartAugmentationDataset). This gave worse results than simply performing augmentation on all samples of the dataset.
 - Since we are using a random sampler, we can ask it to give us as many sample as we want using the given weights. Training on three times our dataset takes too much time, but using twice as much samples did improve the accuracy by around 2%.
 - However, this technique constrains our number of epochs because it effectively doubles training times. 
 - Because of how WeightedRandomSampler works, we have to compute the weight of each sample in the training split. This is not parallelizable (Cf. code comments above) and takes significant time.
 - The largest number of epochs we could thus use is 10.
 - This time, because we can no longer train for that many epochs, using ReduceLROnPlateau leads to better results.

Around that time, we discovered that altough balancing the training data did improve the model's performances, the low scores of our submissions actually came from a witless oversight in how we produced the predicted class. In short, the number given as the output of our model was not directly a class, it was the index of the class in an alphabetically ordered list of all classes. Many thanks to M. Bahl for pointing that out on the course's slack.

After fixing this issue, we now had to deal with a new, quadruple-faced problem. The deadline was approaching, we were running out of GPU time for our last week, the accuracy of our model seemed to have reached a cap, and (almost) everyone else on the leaderboard was sitting between 0.8 and 0.9 accuracy on the leaderboards.

### Final runs : Pretrained Models


Seeing everyone with such similar scores, with some teams having achieved their best score very early in the competition, we deduced that our colleagues were most likely using pre-trained models.

*Why didn't we use pretrained models earlier?*
To be honest, there are probably three main reasons:
 - We wanted to see how good a custom, home-made model could get;
 - We spent around two weeks without knowing how our model would actually perform on the leaderboard;
 - All of the torchvision models are trained on ImageNet, which has 1000 very diverse classes (animals, objects, vegetables, landscapes...) : they are very general models. Our task is much more specific: for example, an overwhelming majority of images in our dataset is mostly green. Therefore, we originally thought that the dataset chosen for this competition was too specific to yield good results on general-purpose, pre-trained models. One of us having a background in NLP led us to be wary of that specificity in our task.

But we were wrong, because pre-trained models performed really, well, in smaller runtimes, than our ResNet. We first tried alexnet with and without oversampling, fine-tuning it or doing feature extraction, and fine-tuning with oversampling yielded better results. (e.g. Alexnet, no oversampling, feature extraction : 0.71. Alexnet, with oversampling, feature extraction : 0.73.)


We then fine-tuned resnet18, resnet152, and vgg19 (with batch normalization), with all of the techniques mentioned above (oversampling, data augmentation, gradient clipping...).

In the end (having no more GPU time nor submissions available), the best result was achieved by:

|*Final performance*| val_acc: 0.82, test score: 0.81 |
|---|:---:|
|Architecture|resnet152 (fine-tuning)|
|input_size|224x224|
|train_proportion|0.8|
|splitting method|sklearn.model_selection.train_test_split|
|Augmentation technique|All classes, random hflip, vflip, resizing crop|
|len(train_dataloader)|len(train_split)|
|batch_size|8|
|epochs|11|
|lr|0.00001|
|grad_clip|5|
|weight_decay|No|
|optimizer|Adam|
|lr scheduler|StepLR with step_size=4|

Key points:
 - The input size is determined by the architecture of resnet152. This leads to the upscaling of some of the images in our dataset, something we never experimented with (Cf. Conclusion)
 - Since we did not have much computing time remaining, we never tried to use more than len(train_split) samples through WeightedRandomSampler.
 - Fine-tuning required a much smaller learning rate than before.
 - Weight decay deteriorated the performance of our model, as it was changing the pre-trained weights.
 - StepLR became the best optimizer again, because ReduceLROnPlateau never activated due to marginal improvements being made every epoch.

## Conclusion

This competition was very enriching, as it covered a lot of aspects of data science: we had to choose a model, we had an unbalanced dataset... If we had to do things differently, or continue working on this project in the future, we would probably focus on the following points:
 - Try larger input sizes with our ResNet: upscaling does not seem to be a problem one should worry about too much.
 - Implement model ensembling. This technique is famous for its use in Kaggle competitions, and could be performed in a number of ways. Averaging over different resnet sizes, over alexnet vgg and resnet152, over our ResNet trained on k-folds of our dataset (cross-validation then model ensembling), or even both at the same time.
 - Look at better, more recent pre-trained models, within torchvision or outside.

# Annex

## Previous Models

In [ ]:
# convolution block avec BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False, kernel_size=3, padding=1, stride=1, pooling_kernel_size=3):
    
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(pooling_kernel_size))
    return nn.Sequential(*layers)

# simpler net
class ConvNet(PlantNetModel):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64) # 3x183x183
        self.conv2 = ConvBlock(64, 128, pool=True) # 128x61x61
        
        
        self.conv3 = ConvBlock(128, 256, pool=True) # 256 x 20 x 20
        self.conv4 = ConvBlock(256, 512, pool=True) # 512 x 6 x 6
        self.conv5 = ConvBlock(512, 512, pool=True) # 512 x 2 x 2
        
        self.dropout = nn.Dropout(p=0.1) 
        
        self.classif = nn.Sequential(nn.MaxPool2d(2),
                                     nn.Flatten(),
                                     nn.Linear(512, n_classes))
        
    def forward(self, batch):
        out = self.conv1(batch)
        out = self.conv2(out)
        out = self.dropout(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.dropout(out)
        out = self.conv5(out)
        out = self.classif(out)
        return out

# resnet architecture
class ResNet(PlantNetModel):
    def __init__(self, in_channels, n_classes):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 32) # 3*183*183
        self.conv2 = ConvBlock(32, 64, pool=True) # 64x61x61
        self.conv3 = ConvBlock(64, 128, pool=True) # 128 x 20 x 20
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128), ConvBlock(128, 128))

        self.conv4 = ConvBlock(128, 256, pool=True) # 256 x 6 x 6
        self.conv5 = ConvBlock(256, 256, pool=True, pooling_kernel_size=2) # 256 x 3 x 3
        self.res2 = nn.Sequential(ConvBlock(256, 256), ConvBlock(256, 256), ConvBlock(256, 256))
        
        self.classif = nn.Sequential(nn.MaxPool2d(3),
                                     nn.Flatten(),
                                     nn.Linear(256, n_classes))
        
        self.dropout = nn.Dropout(p=0.1)
        
    def forward(self, batch):
        out = self.conv1(batch)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.res1(out) + out
        out = self.dropout(out)
        out = self.conv4(out)
        out = self.dropout(out)
        out = self.conv5(out)
        out = self.dropout(out)
        out = self.res2(out) + out
        out = self.classif(out)
        return out

## Code that we no longer use

### "Smart" Data Augmentation

In [ ]:
# when getting from the under-represented classes, transform (to avoid always giving the same images)

class SmartAugmentationDataset(ImageFolder):
    def __init__(self, path, transform=None, aug_class_id=None):
        super().__init__(path, transforms.Compose([transforms.Resize([183, 183]), transforms.ToTensor()]))
        self.transform = transform
        self.aug_class_id = aug_class_id

    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, index): 
        x, y = super().__getitem__(index)
        
        if self.aug_class_id is not None and self.transform is not None: # if the class needs augmentation, and we provided augmentation transforms
            if y in self.aug_class_id:
                x = self.transform(x)
        
        return x, y

In [ ]:
# figure out the indices of classes that need augmentation
aug_class_id = []

for id in range(1, 154):
    n_img = nums.get(str(id))
    if n_img < 927: # 75% of classes (q3)
        aug_class_id.append(id)


In [ ]:
# data augmentation transform
class RandomFlipsTransform:

    def __init__(self, identity_proba=0.5) -> None:
        self.identity_proba = identity_proba

    def __call__(self, img):
        if torch.rand(1) < self.identity_proba:
            return img
        return self.change_image(img)

    def change_image(self, img):
        p = torch.rand(1)
        if p < 0.25:
            return TF.rotate(img, 90)
        elif p < 0.5:
            return TF.rotate(img, 180)
        elif p < 0.75:
            return TF.rotate(img, 270)
        return TF.gaussian_blur(img, 3)
        


aug_transforms = RandomFlipsTransform(0.66)

### Local Manual Data Augmentation

Just to see if it would actually improve performance, we implemented the manual data augmentation on one of our computers. It was not worth it.

In [ ]:
# This code cannot work in a Kaggle notebook due to the read-only nature of the input folder
# also it permanently modifies your input folder, so it should not be ran by default
# also also, throwing out data is very rarely good practice

if False: 
    import os
    from PIL import Image, ImageFilter
    from tqdm.notebook import tqdm

    data_dir = "../input/polytech-nice-data-science-course-2021/polytech/train/"

    classes = os.listdir(data_dir)


    nums = {}
    aug_class_id = []
    big_class_id = []
    for plant_id in classes:
        n = len(os.listdir(data_dir + "/" + plant_id))
        nums[str(plant_id)] = n
        if n < 927:  # q3
            aug_class_id.append(plant_id) # all classes with less than 927 samples will see their size get tripled
        elif n > 3000:
            big_class_id.append(plant_id) # all classes with more than 3000 samples will see their size capped to 3000


    # Augmentation : save a a rotated by 90 degrees version and a slightly blurred version of each image
    # (for classes that need augmentation)
    for plant_id in aug_class_id:
        print(f"Augmenting class {plant_id}...")
        current_path = f"{data_dir}/{plant_id}/"
        current_images = os.listdir(current_path)
        for img_path in tqdm(current_images):
            if img_path[-3:] == "jpg":
                img = Image.open(current_path + img_path)
                tr1 = img.rotate(90, expand=True)
                tr2 = img.filter(filter=ImageFilter.GaussianBlur(radius=1))
                tr1.save(current_path + img_path[:-4] + "r.jpg")
                tr2.save(current_path + img_path[:-4] + "b.jpg")
        print(f"Class {plant_id} now has size {len(os.listdir(current_path))}\n")


    # Remove images from the large classes to bring them back to 3000 images
    for plant_id in big_class_id:
        print(f"Reducing class {plant_id}...")
        current_path = f"{data_dir}/{plant_id}/"
        orig_imgs = os.listdir(current_path)
        orig_s = len(orig_imgs)
        to_remove = orig_s - 3000
        for i in tqdm(range(to_remove)):
            os.remove(current_path + orig_imgs[i])
